In [1]:
!pip install tensorflow
!pip install numpy 
!pip install pandas 
!pip install -U scikit-learn

In [3]:
import pandas as pd
import numpy as np 
import tensorflow as tf
from sklearn.model_selection import train_test_split


In [4]:
def read_dataset(path):
    data=pd.read_csv(path)
    return data 

def get_class_dict(dataset):
    class_list=list(set(dataset['species'].to_list()))
    class_dct={}
    for i in range(len(class_list)):
        class_dct[class_list[i]]=i
    return class_dct

dataset=read_dataset('IRIS.csv')
classes_dict=get_class_dict(dataset)
#Creating copies of dataset so it can be used to obtain 2 values.
d_temp1=dataset.copy()
d_temp2=dataset.copy()
del dataset #Deleting Dataset freeing some space

In [5]:
def label_encoding_dataset(d_temp1,class_dict):
    for i in class_dict.keys():
        d_temp1.loc[d_temp1['species']==i,'species']=class_dict[i]
    
    return d_temp1        

dataset_label_encoded=label_encoding_dataset(d_temp1,classes_dict)
dataset_label_encoded

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,2
1,4.9,3.0,1.4,0.2,2
2,4.7,3.2,1.3,0.2,2
3,4.6,3.1,1.5,0.2,2
4,5.0,3.6,1.4,0.2,2
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1
146,6.3,2.5,5.0,1.9,1
147,6.5,3.0,5.2,2.0,1
148,6.2,3.4,5.4,2.3,1


In [7]:
def one_hot_encoding_dataset(dataset,classes_dict):
    def encode(x,i):
        if x==i:
            return 1
        else:
            return 0
    
    
    def add_columns(dataset,classes_dict):
        dataset_final=dataset.drop(columns=['species'])
        for i in classes_dict.keys():
            dataset_final[i]=dataset['species'].apply(encode,args=(i,))
        return dataset_final
        
    
    dataset_final=add_columns(dataset,classes_dict)
    return dataset_final

dataset_one_hot_encoded=one_hot_encoding_dataset(d_temp2,classes_dict)
dataset_one_hot_encoded

,sepal_length,sepal_width,petal_length,petal_width,Iris-versicolor,Iris-virginica,Iris-setosa
0,5.1,3.5,1.4,0.2,0,0,1
1,4.9,3.0,1.4,0.2,0,0,1
2,4.7,3.2,1.3,0.2,0,0,1
3,4.6,3.1,1.5,0.2,0,0,1
4,5.0,3.6,1.4,0.2,0,0,1
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,1,0
146,6.3,2.5,5.0,1.9,0,1,0
147,6.5,3.0,5.2,2.0,0,1,0
148,6.2,3.4,5.4,2.3,0,1,0


In [9]:
def create_model():
    model=tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=9,activation='sigmoid',input_dim=4))
    model.add(tf.keras.layers.Dense(units=18,activation='sigmoid'))
    model.add(tf.keras.layers.Dense(units=9,activation='sigmoid'))
    model.add(tf.keras.layers.Dense(units=3,activation='softmax'))
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),metrics=['accuracy'])
    #model.summary()
    return model

def get_X_Y_label_encoded(dataset):
    X=np.array(dataset.drop(columns=['species'])).astype(np.float32)
    Y=(np.array(dataset['species']).astype(np.float32)).reshape(-1,1)
    return X,Y

multi_class_model=create_model()
X,Y=get_X_Y_label_encoded(dataset_label_encoded)
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.33, random_state=42)
history_mcm=multi_class_model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=500,batch_size=64)

Epoch 1/500
2/2 [==============================] - 2s 399ms/step - loss: 1.4514 - accuracy: 0.3100 - val_loss: 1.2537 - val_accuracy: 0.3800
Epoch 2/500
2/2 [==============================] - 0s 59ms/step - loss: 1.3222 - accuracy: 0.3100 - val_loss: 1.1855 - val_accuracy: 0.3800
Epoch 3/500
2/2 [==============================] - 0s 55ms/step - loss: 1.2306 - accuracy: 0.3500 - val_loss: 1.1413 - val_accuracy: 0.3200
Epoch 4/500
2/2 [==============================] - 0s 54ms/step - loss: 1.1652 - accuracy: 0.3400 - val_loss: 1.1195 - val_accuracy: 0.3200
Epoch 5/500
2/2 [==============================] - 0s 54ms/step - loss: 1.1169 - accuracy: 0.3400 - val_loss: 1.1139 - val_accuracy: 0.3200
Epoch 6/500
2/2 [==============================] - 0s 66ms/step - loss: 1.1056 - accuracy: 0.3400 - val_loss: 1.1198 - val_accuracy: 0.3200
Epoch 7/500
2/2 [==============================] - 0s 53ms/step - loss: 1.0996 - accuracy: 0.3400 - val_loss: 1.1312 - val_accuracy: 0.3000
Epoch 8/500
2/2 [==

2/2 [==============================] - 0s 68ms/step - loss: 0.3977 - accuracy: 0.9500 - val_loss: 0.3492 - val_accuracy: 0.9800
Epoch 60/500
2/2 [==============================] - 0s 64ms/step - loss: 0.3859 - accuracy: 0.9700 - val_loss: 0.3392 - val_accuracy: 0.9800
Epoch 61/500
2/2 [==============================] - 0s 61ms/step - loss: 0.3749 - accuracy: 0.9700 - val_loss: 0.3289 - val_accuracy: 0.9800
Epoch 62/500
2/2 [==============================] - 0s 63ms/step - loss: 0.3635 - accuracy: 0.9700 - val_loss: 0.3186 - val_accuracy: 0.9800
Epoch 63/500
2/2 [==============================] - 0s 66ms/step - loss: 0.3526 - accuracy: 0.9700 - val_loss: 0.3081 - val_accuracy: 0.9800
Epoch 64/500
2/2 [==============================] - 0s 59ms/step - loss: 0.3417 - accuracy: 0.9800 - val_loss: 0.2984 - val_accuracy: 1.0000
Epoch 65/500
2/2 [==============================] - 0s 60ms/step - loss: 0.3285 - accuracy: 0.9700 - val_loss: 0.2868 - val_accuracy: 0.9800
Epoch 66/500
2/2 [========

2/2 [==============================] - 0s 59ms/step - loss: 0.0821 - accuracy: 0.9800 - val_loss: 0.0780 - val_accuracy: 0.9800
Epoch 118/500
2/2 [==============================] - 0s 58ms/step - loss: 0.0843 - accuracy: 0.9800 - val_loss: 0.0705 - val_accuracy: 0.9800
Epoch 119/500
2/2 [==============================] - 0s 62ms/step - loss: 0.0773 - accuracy: 0.9900 - val_loss: 0.0869 - val_accuracy: 0.9800
Epoch 120/500
2/2 [==============================] - 0s 58ms/step - loss: 0.0787 - accuracy: 0.9800 - val_loss: 0.0779 - val_accuracy: 0.9800
Epoch 121/500
2/2 [==============================] - 0s 60ms/step - loss: 0.0791 - accuracy: 0.9800 - val_loss: 0.0616 - val_accuracy: 0.9800
Epoch 122/500
2/2 [==============================] - 0s 60ms/step - loss: 0.0766 - accuracy: 0.9800 - val_loss: 0.0663 - val_accuracy: 0.9800
Epoch 123/500
2/2 [==============================] - 0s 63ms/step - loss: 0.0762 - accuracy: 0.9900 - val_loss: 0.0804 - val_accuracy: 0.9800
Epoch 124/500
2/2 [=

Epoch 175/500
2/2 [==============================] - 0s 67ms/step - loss: 0.0634 - accuracy: 0.9700 - val_loss: 0.0723 - val_accuracy: 0.9800
Epoch 176/500
2/2 [==============================] - 0s 65ms/step - loss: 0.0597 - accuracy: 0.9800 - val_loss: 0.0472 - val_accuracy: 0.9800
Epoch 177/500
2/2 [==============================] - 0s 69ms/step - loss: 0.0607 - accuracy: 0.9900 - val_loss: 0.0460 - val_accuracy: 0.9800
Epoch 178/500
2/2 [==============================] - 0s 69ms/step - loss: 0.0605 - accuracy: 0.9900 - val_loss: 0.0519 - val_accuracy: 0.9800
Epoch 179/500
2/2 [==============================] - 0s 62ms/step - loss: 0.0620 - accuracy: 0.9800 - val_loss: 0.0704 - val_accuracy: 0.9800
Epoch 180/500
2/2 [==============================] - 0s 64ms/step - loss: 0.0596 - accuracy: 0.9800 - val_loss: 0.0624 - val_accuracy: 0.9800
Epoch 181/500
2/2 [==============================] - 0s 63ms/step - loss: 0.0563 - accuracy: 0.9900 - val_loss: 0.0629 - val_accuracy: 0.9800
Epoch 

Epoch 233/500
2/2 [==============================] - 0s 56ms/step - loss: 0.0522 - accuracy: 0.9900 - val_loss: 0.0477 - val_accuracy: 0.9800
Epoch 234/500
2/2 [==============================] - 0s 47ms/step - loss: 0.0542 - accuracy: 0.9900 - val_loss: 0.0471 - val_accuracy: 0.9800
Epoch 235/500
2/2 [==============================] - 0s 59ms/step - loss: 0.0520 - accuracy: 0.9900 - val_loss: 0.0620 - val_accuracy: 0.9800
Epoch 236/500
2/2 [==============================] - 0s 60ms/step - loss: 0.0505 - accuracy: 0.9900 - val_loss: 0.0769 - val_accuracy: 0.9800
Epoch 237/500
2/2 [==============================] - 0s 59ms/step - loss: 0.0541 - accuracy: 0.9800 - val_loss: 0.0756 - val_accuracy: 0.9800
Epoch 238/500
2/2 [==============================] - 0s 89ms/step - loss: 0.0529 - accuracy: 0.9800 - val_loss: 0.0604 - val_accuracy: 0.9800
Epoch 239/500
2/2 [==============================] - 0s 64ms/step - loss: 0.0498 - accuracy: 0.9900 - val_loss: 0.0476 - val_accuracy: 0.9800
Epoch 

Epoch 291/500
2/2 [==============================] - 0s 64ms/step - loss: 0.0476 - accuracy: 0.9900 - val_loss: 0.0655 - val_accuracy: 0.9800
Epoch 292/500
2/2 [==============================] - 0s 62ms/step - loss: 0.0491 - accuracy: 0.9900 - val_loss: 0.0855 - val_accuracy: 0.9600
Epoch 293/500
2/2 [==============================] - 0s 66ms/step - loss: 0.0505 - accuracy: 0.9800 - val_loss: 0.0661 - val_accuracy: 0.9800
Epoch 294/500
2/2 [==============================] - 0s 65ms/step - loss: 0.0542 - accuracy: 0.9800 - val_loss: 0.0511 - val_accuracy: 0.9800
Epoch 295/500
2/2 [==============================] - 0s 63ms/step - loss: 0.0509 - accuracy: 0.9900 - val_loss: 0.0624 - val_accuracy: 0.9800
Epoch 296/500
2/2 [==============================] - 0s 61ms/step - loss: 0.0470 - accuracy: 0.9900 - val_loss: 0.0590 - val_accuracy: 0.9800
Epoch 297/500
2/2 [==============================] - 0s 65ms/step - loss: 0.0463 - accuracy: 0.9900 - val_loss: 0.0503 - val_accuracy: 0.9800
Epoch 

Epoch 349/500
2/2 [==============================] - 0s 67ms/step - loss: 0.0446 - accuracy: 0.9800 - val_loss: 0.0415 - val_accuracy: 0.9800
Epoch 350/500
2/2 [==============================] - 0s 66ms/step - loss: 0.0513 - accuracy: 0.9900 - val_loss: 0.0424 - val_accuracy: 0.9800
Epoch 351/500
2/2 [==============================] - 0s 62ms/step - loss: 0.0523 - accuracy: 0.9900 - val_loss: 0.0663 - val_accuracy: 0.9800
Epoch 352/500
2/2 [==============================] - 0s 65ms/step - loss: 0.0429 - accuracy: 0.9900 - val_loss: 0.0803 - val_accuracy: 0.9800
Epoch 353/500
2/2 [==============================] - 0s 67ms/step - loss: 0.0452 - accuracy: 0.9800 - val_loss: 0.0729 - val_accuracy: 0.9800
Epoch 354/500
2/2 [==============================] - 0s 91ms/step - loss: 0.0423 - accuracy: 0.9900 - val_loss: 0.0563 - val_accuracy: 0.9800
Epoch 355/500
2/2 [==============================] - 0s 65ms/step - loss: 0.0432 - accuracy: 0.9900 - val_loss: 0.0489 - val_accuracy: 0.9800
Epoch 

Epoch 407/500
2/2 [==============================] - 0s 63ms/step - loss: 0.0454 - accuracy: 0.9900 - val_loss: 0.0639 - val_accuracy: 0.9800
Epoch 408/500
2/2 [==============================] - 0s 57ms/step - loss: 0.0384 - accuracy: 0.9900 - val_loss: 0.0946 - val_accuracy: 0.9600
Epoch 409/500
2/2 [==============================] - 0s 57ms/step - loss: 0.0491 - accuracy: 0.9700 - val_loss: 0.0951 - val_accuracy: 0.9600
Epoch 410/500
2/2 [==============================] - 0s 57ms/step - loss: 0.0427 - accuracy: 0.9700 - val_loss: 0.0577 - val_accuracy: 0.9800
Epoch 411/500
2/2 [==============================] - 0s 54ms/step - loss: 0.0524 - accuracy: 0.9900 - val_loss: 0.0428 - val_accuracy: 0.9800
Epoch 412/500
2/2 [==============================] - 0s 54ms/step - loss: 0.0506 - accuracy: 0.9900 - val_loss: 0.0717 - val_accuracy: 0.9800
Epoch 413/500
2/2 [==============================] - 0s 59ms/step - loss: 0.0396 - accuracy: 0.9900 - val_loss: 0.0916 - val_accuracy: 0.9600
Epoch 

Epoch 465/500
2/2 [==============================] - 0s 58ms/step - loss: 0.0422 - accuracy: 0.9800 - val_loss: 0.0672 - val_accuracy: 0.9800
Epoch 466/500
2/2 [==============================] - 0s 57ms/step - loss: 0.0411 - accuracy: 0.9900 - val_loss: 0.0511 - val_accuracy: 0.9800
Epoch 467/500
2/2 [==============================] - 0s 56ms/step - loss: 0.0429 - accuracy: 0.9900 - val_loss: 0.0626 - val_accuracy: 0.9800
Epoch 468/500
2/2 [==============================] - 0s 58ms/step - loss: 0.0382 - accuracy: 0.9900 - val_loss: 0.0789 - val_accuracy: 0.9800
Epoch 469/500
2/2 [==============================] - 0s 58ms/step - loss: 0.0396 - accuracy: 0.9800 - val_loss: 0.0880 - val_accuracy: 0.9800
Epoch 470/500
2/2 [==============================] - 0s 55ms/step - loss: 0.0407 - accuracy: 0.9800 - val_loss: 0.0708 - val_accuracy: 0.9800
Epoch 471/500
2/2 [==============================] - 0s 59ms/step - loss: 0.0369 - accuracy: 0.9900 - val_loss: 0.0531 - val_accuracy: 0.9800
Epoch 

In [10]:
def predict_output(model,input_user):
    output=model.predict([input_user])
    output=np.argmax(output[0])
    output=list({i for i in classes_dict.keys() if classes_dict[i]==output})
    return output[0]

def get_input_from_users(length):
    input_user=[]
    for i in range(1,length+1):
        input_user.append(float(input(f'Enter {i} ipnut ')))
    return input_user
    
input_1=[5.1,3.5,1.4,0.2]
print(predict_output(multi_class_model,input_1))

input_2=get_input_from_users(dataset_label_encoded.shape[1]-1)
print(predict_output(multi_class_model,input_2))

1/1 [==============================] - 0s 222ms/step
Iris-setosa
Enter 1 ipnut 1
Enter 2 ipnut 2
Enter 3 ipnut 3
Enter 4 ipnut 4
1/1 [==============================] - 0s 75ms/step
Iris-virginica
